In [1]:
from pywinauto_recorder.player import *
import time

Reading configuration file: C:\Users\joao.reis\Pywinauto recorder\config.ini
Window filtering mode: ignore_windows
Ignored windows: []


In [2]:
VNP_PATH = "C:\Program Files\Fortinet\FortiClient\FortiClient.exe"
VNP_PASSWORD = "Pjp072022"
URL_TEST_CONNECTION = 'https://gitlab.sit.trabalho.gov.br/'

In [3]:
def connect_forticlient_vpn():
    send_keys("{VK_LWIN down}r""{VK_LWIN up}")

    with UIPath(u"Run||Window"):
        send_keys("{VK_CONTROL down}a""{VK_CONTROL up}""{DELETE}")
        send_keys('"' + VNP_PATH + '"', pause=0)
        send_keys(u"{ENTER}")

    time.sleep(3)

    with UIPath(u"FortiClient||Pane"):
        click(u"FortiClient -- Zero Trust Fabric Agent||Document", duration=-1)
        send_keys(u"{ENTER}", pause=0) 
        click(u"FortiClient -- Zero Trust Fabric Agent||Document", duration=-1)
        send_keys(u"{TAB}{TAB}{TAB}", pause=0)
        send_keys(VNP_PASSWORD, pause=0)
        send_keys(u"{TAB}{TAB}{ENTER}", pause=0)
        time.sleep(45)

In [4]:
def is_vpn_connected():
    import requests
    from urllib3.exceptions import NewConnectionError

    try:
        x = requests.get(URL_TEST_CONNECTION)
        if x.status_code == 200:
            return True
        else:
            return False
        
    except requests.exceptions.RequestException as error:
        return False

In [5]:
max_attempts = 3

attempts = 0

while True:
    try:
        if is_vpn_connected():
            break
        else:          
            connect_forticlient_vpn()
            assert is_vpn_connected()
            break
    except AssertionError:
        attempts += 1
        print("Falha ao tentar conectar à VPN")
        if attempts >= max_attempts:
            class MyException(Exception):
                pass
            raise MyException("Excedido o número máximo de tentativas de conectar à VPN")